# LAB Assignment 3: Data Collection Using Web Scraping

### Name: Beeta Samad
### Roll Number: 181210016

### Agenda for the Assignment 3
1. Understand the legal and Ethical issue with web-scrapping. In most of the cases scrapping
is legal until you intentionally crash the website or use the data for commercial purposes. You
should check the size of the webiste if you are going to scrape the entire website, use google [1].
You can scrape any kind of data that is publicly available but if the data is copyrighted then
you cannot use it as your own [2].
2. Collect the data from the website http://quotes.toscrape.com/

In [1]:
try:
    from google.colab import drive
    %tensorflow_version 2.x
    COLAB = True
    print("Assignment 3")
    print("Note: using Google CoLab")
except:
    print("Assignment 3")
    print("Note: not using Google CoLab")
    COLAB = False

print("Beeta Samad, 181210016")

Assignment 3
Note: using Google CoLab
Beeta Samad, 181210016


### Task 1:
1. Collect the first 100 quotes from the website http://quotes.toscrape.com/
2. The output should store in three column namely- Quote, Author and Tags.
Hint:
- You may use panda Dataframe framework as pd.DataFrame for storing the data
- use urllib package

In [2]:
!pip install BeautifulSoup4

In [3]:
!pip install pandas

In [4]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd

#defining the headers for scraping:
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
'Accept-Encoding': 'none',
'Accept-Language': 'en-US,en;q=0.8',
'Connection': 'keep-alive'}

In [5]:
#this list stores the links of the author's pages to be later used in the supplimentary problem:
author_link_list = []

def extract_quote_object(div):
    quote_dict = {}
    
    #adding quote and author properties to the dictionary:
    quote_dict['quote'] = div.select(".text")[0].getText()
    quote_dict['author'] = div.select(".author")[0].getText()
    
    #getting the tags list:
    tags = div.select(".tag")
    tags = '; '.join(list(map(lambda tag: tag.getText(), tags)))
    quote_dict['tags'] = tags
    
    #adding the author's link to the lsit:
    author_link_list.append("http://quotes.toscrape.com" + str(div.find_all('a', href=True)[0]['href']))
    
    #returning the dictionary object:
    return quote_dict
    

#defining a function to extract quotes from a particular quote page
def extract_quotes_from_page(page_number, number_of_quotes_required):
    #declaring and initializing necessary link variables, and soup variables (using urllib)
    link = "http://quotes.toscrape.com/page/"+str(page_number)+"/"
    request = Request(link, headers=hdr)
    html = urlopen(request).read().decode()
    soup = BeautifulSoup(html, "html.parser")
    
    #select all divs containing the class 'quote':
    quotes = soup.select(".quote")
    
    #mapping each of the div's contents to a list of dictionaries 
    #(each dictionary has two keys named 'quote' and 'author'):
    all_the_quotes = []
    quotes = list(map(extract_quote_object, quotes));
    
    #initializing how many quotes to select from the current page:
    number_of_quotes_from_current_page = 10 if number_of_quotes_required>10 else number_of_quotes_required
    
    #returning the quotes required from this page, as well as the number of quotes further required :
    return quotes[0:number_of_quotes_from_current_page], (number_of_quotes_required-len(quotes))

In [6]:
#declaring an empty all_quotes list that will store all the quotes:
all_quotes = []
#declaring and initializing a variable the tracks the number of quotes required
number_of_quotes_required = 100
#declaring a variable to keep track of the page number that we're currently scraping:
current_page_number = 1

#while loop to go through each page and extract quotes from each page onto the all_quotes lsit:
while number_of_quotes_required > 0:
    quotes_of_current_page, number_of_quotes_required = extract_quotes_from_page(current_page_number, number_of_quotes_required)
    current_page_number += 1
    all_quotes.extend(quotes_of_current_page)

In [7]:
data_frame = pd.DataFrame(all_quotes)
data_frame

,quote,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,change; deep-thoughts; thinking; world
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,abilities; choices
2,“There are only two ways to live your life. On...,Albert Einstein,inspirational; life; live; miracle; miracles
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,aliteracy; books; classic; humor
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,be-yourself; inspirational
...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,better-life-empathy
96,“You have to write the book that wants to be w...,Madeleine L'Engle,books; children; difficult; grown-ups; write; ...
97,“Never tell the truth to people who are not wo...,Mark Twain,truth
98,"“A person's a person, no matter how small.”",Dr. Seuss,inspirational


### Supplimentry Problem :
1. Add two more column of Date of Birth (DoB) and Place of Birth(PoB) of the Author to the output

In [8]:
import requests

for index, link in enumerate(author_link_list):
    try:
        res = requests.get(link)
        soup = BeautifulSoup(res.text,"html.parser")
        all_quotes[index]['DoB'] = soup.select(".author-born-date")[0].getText()
        all_quotes[index]['PoB'] = soup.select(".author-born-location")[0].getText().replace('in ', '')
    except:
        print("Error trying to get info from the author of index", index)
    
all_quotes

[{'DoB': 'March 14, 1879',
  'PoB': 'Ulm, Germany',
  'author': 'Albert Einstein',
  'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'tags': 'change; deep-thoughts; thinking; world'},
 {'DoB': 'July 31, 1965',
  'PoB': 'Yate, South Gloucestershire, England, The United Kingdom',
  'author': 'J.K. Rowling',
  'quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'tags': 'abilities; choices'},
 {'DoB': 'March 14, 1879',
  'PoB': 'Ulm, Germany',
  'author': 'Albert Einstein',
  'quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'tags': 'inspirational; life; live; miracle; miracles'},
 {'DoB': 'December 16, 1775',
  'PoB': 'Steventon Rectory, Hampshire, The United Kingdom',
  'author': 'Jane Austen',
  'quote': '“The person, be it gentleman or lady, who has not pleasure i

In [9]:
data_frame = pd.DataFrame(all_quotes)
data_frame

,quote,author,tags,DoB,PoB
0,“The world as we have created it is a process ...,Albert Einstein,change; deep-thoughts; thinking; world,"March 14, 1879","Ulm, Germany"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,abilities; choices,"July 31, 1965","Yate, South Gloucestershire, England, The Unit..."
2,“There are only two ways to live your life. On...,Albert Einstein,inspirational; life; live; miracle; miracles,"March 14, 1879","Ulm, Germany"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,aliteracy; books; classic; humor,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,be-yourself; inspirational,"June 01, 1926",The United States
...,...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,better-life-empathy,"April 28, 1926","Monroeville, Alabama, The United States"
96,“You have to write the book that wants to be w...,Madeleine L'Engle,books; children; difficult; grown-ups; write; ...,"November 29, 1918","New York City, New York, The United States"
97,“Never tell the truth to people who are not wo...,Mark Twain,truth,"November 30, 1835","Florida, Missouri, The United States"
98,"“A person's a person, no matter how small.”",Dr. Seuss,inspirational,"March 02, 1904","Springfield, MA, The United States"
